
# EP1 de Programacao Paralela e distribuida - MAC5742



| Nome | N° USP |
|------|------|
| Ariel Serranoni Soares da Silva| 7658024 |
| Thiago Martinoni | 9793401 |


## Resumo

Em matemática, fractais são conjuntos são auto-similares. Intuitivamente, essa propriedade significa que, se $X$ é um conjunto
auto-similar, então $X$ é formado por cópias identicas a si mesmo. Não raramente a representação gráfica (plotagem) de fractais

Em particular, o conjunto de mandelbrot é um fractal que pode ser plotado facilmente utilizando apenas conceitos básicos de programação, como condicionais e laços.
A implementação sequencial  é muito comum em disciplinas introdutórias de computação. Neste trabalho vamos além, implementaremos três versões deste algorítmo, a clássica sequencial, uma utilizando pthreads e uma terceira utilizando open mp. Os códigos em C e seus encontram-se disponíveis em bla. Aqui vamos analisar o desempenho de cada algorítmo afim de obter um parâmetro de comparação a respeito do desempenho dessas bibliotecas. 

## Implementações em C

Temos três arquivos, cada um responsável por calcular o conjunto de mandelbrot utilizando um método diferente. 
Como o programa `mandelbrot_seq` serve como base para os outros dois, é natural que comentemos algumas das mudanças
que fizemos na implementação do primeiro programa antes de continuarmos: 

#### Percorrendo vetores multidimensionais com um único laço

Na implementação original que recebemos como base, a função `compute_mandelbrot()` operava com dois laços. Neste contexto,
cada iteração do laço externo era responsável por uma linha da imagem. Portanto, para dividir o trabalho do algorítmo em unidades menores do que uma linha, seríamos obrigados a criar threads dentro de threads ou então partir para um novo 
paradigma de implementação do nosso programa. 

A mudança que fizemos foi considerar um único parâmetro `i`, desta vez percorrendo todos os `image_buffer_size` pixels da imagem. Sabendo o número `image_size` de pixels em cada linha, conseguimos descobrir a linha (`i/image_size`) e coluna (`i%image_size`) exatas de um pixel a partir de um único parâmetro. 

Vale notar que esse mesmo raciocínio pode ser generalizado para vetores com mais dimensões (tensores). Por exemplo, 
adicionando uma terceira dimensão, consegueríamos saber a "altura" a qual o índice `i` corresponde ao fazer o cálculo
`i/image_size ^2`.

Além disso, também é importante enxergar como esta mudança contribui para a paralelização do algorítmo. Como citamos acima,
a divisão do trabalho em unidades tão pequenas quanto possível nos oferece uma flexibilidade maior ao alocar tarfas para cada
thread.


#### Recebendo mais opções pela linha de comando

Além da mudança descrita anteriormente, também adicionamos um argumento extra `i_o` a ser recebido na inicialização do programa. Dessa forma, conseguimos mudar esta opção no programa sem alterar o código, contribuindo para a praticidade na realização do experimento.

#### Cálculos de tempo 

Para medir o desempenho de um programa, é essensial que tenhamos uma maneira de medir o tempo de execução do mesmo. 
Para isso, aproveitamos a struct `timer_info` do miniEP3 e encapsulamos as medições nas funçoes `start_timer()` e `stop_timer()`. Também retornamos os resultados utilizando os mesmos métodos no miniEP3 através da função `print_results()`

### Implementações em paralelo

Nas implementações em paralelo, utilizamos a estratégia de divisão estática tanto no programa `mandelbrot_pth` quanto no `mandelbrot_omp` afim de obter uma comparação mais justa possível. Apesar disso, não sabemos exatamente o que OpenMp faz com
o resto da divisão das unidades de trabalho pelo número de threads. Enquanto isso, na implementação utilizando PThreads adicionamos este excedente ao trabalho da primeira thread.



## Infra estrutura Julia

Para este relatório, também reaproveitamos muito da estrutura dos trabalhos anteriores. Utilizamos Jupyter notebooks e Julia na versão 1.3.0. Também usamos os mesmos pacotes do nosso último trabalho. A seguinte célula fará as instalações necessárias dos pacotes Julia:  

In [1]:
]up

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `/var/fasttmp/parallel-mandelbrot/src/Project.toml`
 [no changes]
  Updating `/var/fasttmp/parallel-mandelbrot/src/Manifest.toml`
 [no changes]


Antes de iniciarmos os experimentos, vale conferir se tudo correu como esperado na instalação acima.

In [2]:
] st

    Status `/var/fasttmp/parallel-mandelbrot/src/Project.toml`
  [336ed68f] CSV v0.6.2
  [a93c6f00] DataFrames v0.21.1
  [31c24e10] Distributions v0.23.3
  [7073ff75] IJulia v1.21.2
  [8314cec4] PGFPlotsX v1.2.6
  [1a8c2f83] Query v0.12.2
  [f3b207a7] StatsPlots v0.14.6


### Experimentos e Coleta de Dados

O Primeiro passo para analisarmos o tempo de execução de um determinado programa sempre foi e sempre será executar o programa
em questão. A função abaixo é uma adaptação da função `measure_monte_carlo()` do mini EP3. Seu objetivo é justamente fazer uma chamada de um dos programas em C na linha de comando e salvar os dados que serão relevantes para nossa análise.


In [3]:
using DataFrames, Query, StatsPlots, Statistics

function mandelbrot(method, region, image_size, i_o, threads=1)
    a=region[2][1]
    b=region[2][2]
    c=region[2][3]
    d=region[2][4]
    results = parse.(Float64,
        split(chomp(read(`./mandelbrot_$method $a  $b $c $d $image_size $i_o $threads`, String)), ", "))
        
    return DataFrame(size = image_size,
                region=region[1],
                delta_x = region[2][2] - region[2][1],
                delta_y = region[2][4] - region[2][3],
                duration_clock = results[1],
                duration_mono = results[2],
                duration_sys = results[3],
                threads=threads,
                method=method,
                i_o=i_o)
    end

mandelbrot (generic function with 2 methods)

Para que nossa análise tenha algum valor do ponto de vista estatístico, precisamos nos basear em uma amostra consideravelmente
grande. A função a seguir, também adaptada do miniEP3 tem o intuito de automatizar 
repetidas chamadas dos programas em C, feitas pela função acima. A função `run_experiments()` a seguir nos permite variar
todos os parâmetros das chamadas feitas pela função acima e estabelecer um número de repetições. Então as chamadas são executadas, os resultados coletados e retornados na forma de um `DataFrame`.


In [4]:
function run_experiments(methods, regions ,image_size, threads, i_o, repetitions)
    run(`make`)
    
    results = DataFrame(size = Int64[],
                region = String[],
                delta_x = Float64[],
                delta_y = Float64[],
                duration_clock = Float64[],
                duration_mono = Float64[],
                duration_sys = Float64[],
                threads= Int64[],
                method= String[],
                i_o = Int64[])
        
    for tuple in regions
        for method in methods
            for size in image_size
                for t in 1:length(threads)
                    if ( threads[t] <= 1  || method != "seq") 
                            if(method == "seq")
                                for alt in i_o
                                    for r in 1:repetitions
                                        append!(results, mandelbrot(method, tuple, size, alt, threads[t]))
                
                                    end
                                end
                            end
                            else
                                for r in 1:repetitions
                                        append!(results, mandelbrot(method, tuple, size, 0, threads[t]))
                                end
                            end
                        end
                    end
                end
            end

    return results
end

run_experiments (generic function with 1 method)

In [5]:
function parse_results(results)
  parsed_results=results|>
                @groupby((_.threads,_.method,_.region, _.size)) |>
                @map({threads_method = key(_),
                repetitions=repetitions,
                i_o= i_o,
                delta_x = mean(_.delta_x),
                delta_y = mean(_.delta_y),
                mean_duration_clock = mean(_.duration_clock),
                mean_duration_mono = mean(_.duration_mono),
                mean_duration_sys = mean(_.duration_sys),
                ci_duration_clock = 1.96 * std(_.duration_clock),
                ci_duration_mono = 1.96 * std(_.duration_mono),
                ci_duration_sys = 1.96 * std(_.duration_sys)})|>
                DataFrame
    
      final_results = DataFrame(i_o=Int64[],
                    threads= Int64[],
                    method= String[],
                    region = String[],
                    size = Int64[],
                    delta_x = Float64[],
                    delta_y = Float64[],
                    mean_duration_clock = Float64[],
                    mean_duration_mono = Float64[],
                    mean_duration_sys = Float64[],
                    ci_duration_clock = Float64[],
                    ci_duration_mono = Float64[],
                    ci_duration_sys = Float64[],
                    repetitions = Int64[])
    
        for i in 1:length(parsed_results.threads_method)
            
            append!(final_results, DataFrame(size=parsed_results.threads_method[i][4],
                                        delta_x=parsed_results.delta_x[i],
                                         delta_y=parsed_results.delta_y[i],
                                        mean_duration_clock=parsed_results.mean_duration_clock[i],
                                        mean_duration_mono=parsed_results.mean_duration_mono[i],
                                        mean_duration_sys=parsed_results.mean_duration_sys[i],
                                        ci_duration_clock=parsed_results.ci_duration_clock[i],
                                        ci_duration_mono=parsed_results.ci_duration_mono[i],
                                        ci_duration_sys=parsed_results.ci_duration_sys[i],
                                        repetitions =parsed_results.repetitions[i],
                                        threads = parsed_results.threads_method[i][1],
                                        method = parsed_results.threads_method[i][2],
                                        region = parsed_results.threads_method[i][3],
                                        i_o = parsed_results.i_o[i]))
    
        end
    return final_results
end

parse_results (generic function with 1 method)

Antes de iniciar os experimentos, vamos ajustar parâmetros para que os experimentos ocorram como proposto no enunciado.


|               	| Sequencial         	| Pthreads & OpenMp  	| 
|----------------	|--------------------	|--------------------	|
| I/O e Alocação 	| Com e Sem          	| Sem                	|
| N° de Threads  	| 1                  	| $2^0\ldots 2^5$    	|
| Lado da Imagem 	| $2^0\ldots 2^{13}$ 	| $2^0\ldots 2^{13}$ 	|
| Região         	| Todas              	| Todas              	|


In [6]:
sizes=[2^x for x in 4:13]
threads = [2 ^ x for x in 0:5]
methods = ["pth","omp","seq"]
i_o=[0,1]
regions = Dict("Full Picture" => (-2.5,1.5,-2.0,2.0),
                "Seahorse Valley" => (-0.8,-0.7,0.05,0.15),
                "Elephant Valley" => (0.175,0.375,-0.1,0.1),
                "Triple Spiral Valley" =>(-0.188,-0.012,0.554,0.754))
repetitions=10

10

Como a função run_experiments pode fazer muitas chamadas dos programas em C, salvar os resultados em um arquivo pode ser uma forma
de agilizar o fluxo de trabalho. No nosso caso, precisamos executar umas 1200 chamadas na linha de comando. Isso torna essencial
salvar os resultados. As seguintes funções são reaproveitadas do miniEP3 e nos ajudam com esse trabalho.

In [7]:
using CSV

function save_csv_results(data_frame)
    CSV.write("./data_frame.csv",data_frame,delim='\t')
end

function read_csv_results()
    df=CSV.read("./data_frame.csv")
    return df
end
results = run_experiments(methods, regions, sizes, threads, i_o, repetitions)
parsed_results = parse_results(results)
save_csv_results(parsed_results)

make: Nothing to be done for 'all'.


"./data_frame.csv"

In [11]:
parsed_results

,i_o,threads,method,region,size,delta_x,delta_y,mean_duration_clock
,Int64,Int64,String,String,Int64,Float64,Float64,Float64
1,0,1,seq,Full Picture,16,4.0,4.0,0.00023515
2,1,1,seq,Full Picture,16,4.0,4.0,0.00023515
3,0,2,seq,Full Picture,16,4.0,4.0,0.0001096
4,1,2,seq,Full Picture,16,4.0,4.0,0.0001096
5,0,4,seq,Full Picture,16,4.0,4.0,0.0001093
6,1,4,seq,Full Picture,16,4.0,4.0,0.0001093
7,0,8,seq,Full Picture,16,4.0,4.0,0.0001123
8,1,8,seq,Full Picture,16,4.0,4.0,0.0001123
9,0,16,seq,Full Picture,16,4.0,4.0,0.0001105


### Vizualização e Análise dos Resultados

A função a seguir será de grande valia para simplificar a geração de gráficos

In [13]:
pgfplotsx()

function plot_results(x, y, group, target_label, series_label, hline = [], yerror = [], max_thread_power = 10)
    if yerror != []
        s = scatter(x,
            y,
            xaxis = :log2,
            group=group,
            xlabel = "Threads",
            xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror,
            alpha = 100,
            labels = series_label,
            legend = :topright)
    else
        p = scatter(x,
            y,
            ,
            group=group,
            xlabel = "Threads",
            xticks = [2 ^ x for x in 0:max_thread_power],
            alpha = 0.6,
            labels = series_label,
            legend = :topright)
    end

    if hline != []
        plot!(x,
            hline,
            xaxis = :log2,
            xlabel = "Threads",
            xticks = [2 ^ x for x in 0:max_thread_power],
            labels = target_label,
            line = :dash,
            width = 2.0)
    end

    return p
end

plot_results (generic function with 4 methods)

In [24]:
parsed_results[parsed_results.method .== "seq",:]

,i_o,threads,method,region,size,delta_x,delta_y,mean_duration_clock
,Int64,Int64,String,String,Int64,Float64,Float64,Float64
1,0,1,seq,Full Picture,16,4.0,4.0,0.00023515
2,1,1,seq,Full Picture,16,4.0,4.0,0.00023515
3,0,2,seq,Full Picture,16,4.0,4.0,0.0001096
4,1,2,seq,Full Picture,16,4.0,4.0,0.0001096
5,0,4,seq,Full Picture,16,4.0,4.0,0.0001093
6,1,4,seq,Full Picture,16,4.0,4.0,0.0001093
7,0,8,seq,Full Picture,16,4.0,4.0,0.0001123
8,1,8,seq,Full Picture,16,4.0,4.0,0.0001123
9,0,16,seq,Full Picture,16,4.0,4.0,0.0001105


In [14]:
## Little data wrangling

data_full_picture = parsed_results[parsed_results.region .== "Full Picture",:]
data_seahorse_valley = parsed_results[parsed_results.region .== "Seahorse Valley" ,:]
data_elephant_valley = parsed_results[parsed_results.region .== "Elephant Valley", :]
data_triple_spiral = parsed_results[parsed_results.region .== "Triple Spiral Valley", :]

##Plots
p1=plot_results(data_full_picture.threads,
            data_full_picture.mean_duration_mono,
            data_full_picture.method,
            data_full_picture.method,
            max_thread_power = 5)

p2=plot_results(data_seahorse_valley.threads,
            data_seahorse_valley.mean_duration_mono,
            data_seahorse_valley.method,
            "",
            data_seahorse_valley.method,
            max_thread_power = 5)

p3=plot_results(data_elephant_valley.threads,
                data_elephant_valley.mean_duration_mono,
                data_elephant_valley.method,
                data_elephant_valley.method,
                max_thread_power=5)

##Layout
plot(p1,p2,p3,p4 layout=(2,2), title = "Qualquer coisa" , legend=true)

MethodError: MethodError: no method matching plot_results(::Array{Int64,1}, ::Array{Float64,1}, ::Array{String,1}, ::Array{String,1}; max_thread_power=5)
Closest candidates are:
  plot_results(::Any, ::Any, ::Any, ::Any, !Matched::Any; hline, yerror, max_thread_power) at In[13]:4
  plot_results(::Any, ::Any, ::Any, ::Any, !Matched::Any, !Matched::Any) at In[13]:4 got unsupported keyword argument "max_thread_power"
  plot_results(::Any, ::Any, ::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any) at In[13]:4 got unsupported keyword argument "max_thread_power"
  ...

## Observações finais

### Infra Estrutura de trabalho remoto utilizando SSH

A realização deste trabalho demandou muito poder computacional. Como nenhum dos elementos da dupla possui um computador poderoso o suficiente para realizar testes e ainda viver, precisamos aprender um pouco sobre o uso remoto da rede IME. Como
nesse momento em que a grande maioria da comunidade IME não consegue estar fisicamente presente no instituto, resolvemos fazer
uma pequena descrição de como utilizamos a máquina brucutu da rede IME para realizar nossos experimentos.

Em primeiro lugar, vale dizer que o acesso às redes do IME

#### Incluindo um novo kernel


#### Abrindo uma porta para acesso local